In [1]:
import gct,os
import numpy as np

In [2]:
names = sorted([u for u in gct.list_local_graph() if u.startswith("FIXED_SQRT_ud_wu_N")], key=lambda u: int(u.split("_")[4][1:]))
len(names)

48

In [3]:
names[:2]

['FIXED_SQRT_ud_wu_N5_C5_in2_it1', 'FIXED_SQRT_ud_wu_N6_C6_in3_it1']

In [4]:
names[:3]

['FIXED_SQRT_ud_wu_N5_C5_in2_it1',
 'FIXED_SQRT_ud_wu_N6_C6_in3_it1',
 'FIXED_SQRT_ud_wu_N7_C7_in3_it1']

In [5]:
def run_alg(algname):
    for dataname in names:
        print("running", algname,dataname)
        retcode = os.system("timeout -k 1h 1h python run_alg.py {} {} 0 >  /data/log/{}_{}.log 2>&1".format(algname,dataname, algname,dataname))
        if retcode !=0:
            break


In [6]:
import numpy as np
algs =gct.list_algorithms()
np.random.shuffle(algs)
len(algs)

79

run_alg("alg_GossipMap")

run_alg("alg_pg_label_propagation")

run_alg("cdc_EAGLE")

In [21]:
from joblib import Parallel, delayed
ret=Parallel(n_jobs=6)(delayed(run_alg)(x) for x in algs)    

In [14]:
algs={'cdc_DEMON',
 'cdc_HDEMON',
 'karateclub_BigClam',
 'karateclub_DANMF',
 'karateclub_EdMot',
 'karateclub_EgoNetSplitter',
 'karateclub_NNSED',
 'karateclub_SCD',
 'karateclub_SymmNMF'}
from joblib import Parallel, delayed
ret=Parallel(n_jobs=3)(delayed(run_alg)(x) for x in algs)    

In [22]:
import numpy as np
np.where(not ret), len(ret)

((array([], dtype=int64),), 72)

In [15]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if "FIXED_SQRT_ud_wu_N" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            results[u + run] = a

len(results)

3047

In [16]:
import pandas as pd
df = pd.DataFrame([(u['runname'],u['dataname'],u['timecost']) for u in results.values()], columns=['alg','datanem','timecost'])

In [17]:
missedargs = set(gct.list_algorithms()).difference(df['alg'].values)
missedargs


set()

In [18]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (df.sort_values("timecost"))

,alg,datanem,timecost
44,igraph_community_label_propagation,FIXED_SQRT_ud_wu_N5_C5_in2_it1,0.000093
53,igraph_community_fastgreedy,FIXED_SQRT_ud_wu_N5_C5_in2_it1,0.000098
122,igraph_community_label_propagation,FIXED_SQRT_ud_wu_N6_C6_in3_it1,0.000113
200,igraph_community_label_propagation,FIXED_SQRT_ud_wu_N7_C7_in3_it1,0.000123
140,igraph_community_multilevel,FIXED_SQRT_ud_wu_N6_C6_in3_it1,0.000124
...,...,...,...
1615,cdc_ParCPM,FIXED_SQRT_ud_wu_N44_C44_in22_it11,3384.257661
2806,karateclub_DANMF,FIXED_SQRT_ud_wu_N209_C209_in104_it52,3399.768246
2798,cdc_MOSES,FIXED_SQRT_ud_wu_N209_C209_in104_it52,3412.327678
2427,networkit_CutClustering,FIXED_SQRT_ud_wu_N121_C121_in60_it30,3495.735493


In [19]:
df['N']=df['datanem'].map(lambda u: int(u.split("_")[-4][1:])*int(u.split("_")[-3][1:]))

In [20]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    pdf=df.pivot_table(columns=['N'], index=['alg'], values='timecost')
    for x in missedargs:
        pdf=pdf.append(pd.Series(name=x))

    display(pdf)

N,25,36,49,64,81,100,121,144,169,196,256,289,361,441,529,625,784,900,1089,1369,1600,1936,2304,2809,3364,4096,4900,5776,7056,8464,10201,12100,14641,17424,21025,25281,30276,36481,43681,52441,63001,75625,90601,108900,131044,156816,188356,225625
alg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
alg_GossipMap,0.637229,0.568937,0.666936,0.606437,0.608052,0.636176,0.701309,0.682027,0.655246,0.670901,0.654093,0.671394,0.656108,0.668355,0.708730,0.708182,0.721268,0.748649,0.813690,0.914399,0.914113,1.173813,1.341813,1.580528,1.953952,2.449259,3.095835,3.908968,4.957160,6.565072,8.597775,11.452945,14.074097,18.220700,27.248082,35.623761,48.650345,63.264887,81.849023,110.329313,147.581029,196.131395,270.561004,341.832089,NaN,NaN,NaN,NaN
alg_Paris,0.011692,0.005680,0.008004,0.012484,0.016367,0.022786,0.028424,0.041332,0.049802,0.062604,0.098971,0.112674,0.154605,0.226236,0.285047,0.389963,0.572218,0.704841,0.952856,1.374818,1.795039,2.412730,3.228182,4.380702,5.792206,8.169003,10.599506,14.852348,20.401580,26.653726,36.928005,47.348542,65.114353,87.341035,114.230687,154.716085,209.057275,284.765139,387.504221,521.718897,689.891660,894.283533,1269.524457,1750.220973,NaN,NaN,NaN,NaN
alg_RelaxMap,0.016578,0.013746,0.014648,0.015240,0.014791,0.015321,0.018549,0.017663,0.021666,0.023287,0.028771,0.038374,0.042457,0.049912,0.056584,0.070387,0.092519,0.106312,0.133578,0.179288,0.229640,0.314054,0.401046,0.536142,0.672371,0.953194,1.233047,1.597793,2.138680,2.885486,3.759473,4.850763,6.666070,8.856367,11.841577,15.712035,21.917946,28.605123,37.665655,52.205357,68.369132,92.854162,122.291645,159.528832,NaN,NaN,NaN,NaN
alg_lso_cluster,0.027627,0.015512,0.015421,0.019785,0.016526,0.016327,0.017358,0.017255,0.017527,0.018289,0.019190,0.020309,0.024207,0.023617,0.026384,0.029680,0.039224,0.040409,0.059338,0.063499,0.077919,0.103244,0.123020,0.163516,0.244061,0.313017,0.435395,0.508679,0.785170,1.009689,1.211585,1.829958,2.348699,3.145318,2.977255,5.738005,7.765262,9.727805,13.437730,17.714014,24.833503,34.452149,41.525937,56.115514,NaN,NaN,NaN,NaN
alg_pg_label_propagation,0.569591,0.397617,0.390266,0.383585,0.392378,0.389780,0.393001,0.390974,0.387339,0.393827,0.396061,0.389661,0.395823,0.396214,0.396315,0.401070,0.398928,0.397920,0.402517,0.407955,0.404352,0.421164,0.447215,0.481014,0.534180,0.606486,0.679231,0.791456,0.928784,1.122698,1.374241,1.662645,2.117681,2.698943,3.450336,4.488817,5.907513,7.831851,10.287578,13.591146,18.413641,23.486844,31.093556,41.598476,NaN,NaN,NaN,NaN
alg_streamcom,0.011772,0.010573,0.011240,0.011546,0.011759,0.011438,0.011627,0.011618,0.011601,0.012374,0.013343,0.013117,0.011085,0.011333,0.011186,0.014429,0.013943,NaN,0.012167,0.019197,0.017427,0.017637,0.015367,0.033427,0.028711,0.025238,0.018776,0.075976,0.077517,0.050912,NaN,0.204683,0.222332,0.125912,0.124979,0.695500,0.709451,NaN,1.502395,1.281740,1.352333,NaN,4.920745,4.548273,NaN,NaN,NaN,NaN
cdc_CONGA,0.129677,0.133960,0.119834,0.130683,0.143349,0.150156,0.156401,0.184008,0.178565,0.212419,0.226339,0.284798,0.268434,0.311834,0.334497,0.463618,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cdc_CliquePercolation,0.015275,0.012361,0.009816,0.009731,0.009869,0.010239,0.010893,0.012028,0.012741,0.013961,0.017459,0.018526,0.023115,0.029237,0.036212,0.047318,0.070475,0.089777,0.117685,0.177375,0.244078,0.350434,0.490354,0.674875,0.987350,1.523786,2.150956,3.049948,7.695590,11.069819,15.879874,24.744471,39.024525,50.285291,75.598235,97.979095,182.744128,334.153769,381.188246,653.131696,592.894855,963.936403,1617.928435,1724.571307,NaN,NaN,NaN,NaN
cdc_Connected_Iterative_Scan,0.016461,0.013621,0.015562,0.018084,0.020307,0.024394,0.030672,0.044949,0.056277,0.070764,0.127978,0.151591,0.222724,0.371096,0.527338,0.819960,1.325860,1.728549,2.836141,4.935606,7.592370,12.182712,19.908769,36.872775,58.808600,108.449975,171.420753,270.801755,452.296196,723.306046,1141.014827,1685.075775,2825.076282,